In [15]:
import numpy as np
import math

In [16]:
def scale(x, bitwidth, sign):

  z = 0

  if(sign):
    s = 2 * max(np.abs(np.max(x)),np.abs(np.min(x))) /(2**bitwidth-1)

  else:
    s = np.max(np.abs(x))/(2**bitwidth-1)

  return s,z

In [17]:
def quantize(x,s,z,b, sign):
  if (sign):
    min= -(2**(b-1))
    max= (2**(b-1))-1
  else:
    min=0
    max= (2**b)-1

  x_int= np.clip(np.round(x/s)+z, min, max).astype(np.int32)
  return x_int

In [18]:
import torch
print("Is CUDA available: ", torch.cuda.is_available())

Is CUDA available:  True


In [19]:
x=np.linspace(-5,6,100000)
print(x)
s, z = scale(x, 8, True)
x_int= quantize(x,s,z,8, True)
print(x_int)
print(np.sum(np.abs(x_int)<3))

[-5.      -4.99989 -4.99978 ...  5.99978  5.99989  6.     ]
[-106 -106 -106 ...  127  127  127]
2139


In [21]:
def assign_PE (N_ROWS_ARRAY, N_COLS_ARRAY, f_weights):

  # Quantizing weights

  bitwidth = 8
  sign = True
  s, z = scale(f_weights, bitwidth, sign)
  #Q_f_weights = quantize(f_weights, s, z, bitwidth, sign)
  Q_f_weights = f_weights
  print(np.sum(abs(Q_f_weights) < pow(2,-8)))
#-----------------------------------------------------------------------------------------------
  # Sorting filters based on numbers of zeros they have.(ascending order of zero density)

  near_zero_count = 0
  near_zero_threshold = pow(2,-8)
  near_zero_count = np.sum(np.abs(Q_f_weights) < near_zero_threshold, axis = (0, 1, 2))
  #print(near_zero_count)
  sorted_filter_index = np.argsort(near_zero_count)
  print(sorted_filter_index)
  #f_weights_sorted = Q_f_weights[:,:,:,sorted_filter_index]
  f_weights_sorted = Q_f_weights
  #print(f_weights_sorted)

#-----------------------------------------------------------------------------------------------
  # Saving indices of each elements of sorted weights.

  indices_of_elements_in_f_weights_sorted = np.stack(np.indices(f_weights_sorted.shape), axis=-1)
  #print(indices_of_elements_in_f_weights_sorted)

#-----------------------------------------------------------------------------------------------
  # Compressing filters.

  if f_weights.shape[2]*f_weights.shape[0] <= N_ROWS_ARRAY:

    n = 1

  else:

    n = math.ceil(f_weights.shape[2]/math.floor(N_ROWS_ARRAY/f_weights.shape[0]))
  print(f_weights_sorted.shape[3])
  print(f_weights_sorted.shape[2])
  print(f_weights_sorted.shape[1])
  print(f_weights_sorted.shape[0])
  print(n)
  remaining_array = N_COLS_ARRAY * np.ones((N_ROWS_ARRAY, 1))
  full_array = N_COLS_ARRAY * np.ones((N_ROWS_ARRAY, 1))
  num_of_col= np.zeros((N_ROWS_ARRAY,1))
  n_columns = np.zeros((N_ROWS_ARRAY,N_COLS_ARRAY))
  sel_mux_tr = np.zeros((N_ROWS_ARRAY,N_COLS_ARRAY))
  f_s = np.zeros((N_ROWS_ARRAY,N_COLS_ARRAY))
  weights_array = np.zeros((N_ROWS_ARRAY,N_COLS_ARRAY))
  en_adder_node =  np.zeros((N_ROWS_ARRAY,N_COLS_ARRAY))
  result_col = np.zeros((1,N_COLS_ARRAY))
  PE_array_weights = []
  PE_array_weights.append([])
  PE_array_index= []
  PE_array_index.append([])
  PE_array_f_sel = []
  PE_array_f_sel.append([])
  PE_array_filter_num= []
  PE_array_filter_num.append([])
  num_of_columns =[]
  num_of_columns.append([])
  sel_mux_transfer= []
  sel_mux_transfer.append([])
  en_adder_node_all = []
  en_adder_node_all.append([])
  result_columns= []
  result_columns.append([])
  count = 0
  indexes=[]
  f_sels = []
  weights = []


  for ch_slices_num in range(n):

    for filter_num in range(f_weights.shape[3]):

      for ch_num in range((ch_slices_num)*math.floor(N_ROWS_ARRAY/f_weights.shape[0]),(ch_slices_num+1)*math.floor(N_ROWS_ARRAY/f_weights.shape[0])):

        if ch_num < f_weights.shape[2] :

          num_of_col[f_weights.shape[0]*(ch_num % math.floor(N_ROWS_ARRAY/f_weights.shape[0]) ) : f_weights.shape[0]*(1 + ch_num % math.floor(N_ROWS_ARRAY/f_weights.shape[0])) , :], index, f_select, weight = filter_assign(f_weights_sorted[:,:,ch_num, filter_num], near_zero_threshold, f_weights.shape[0] )
          indexes.append(index)
          f_sels.append(f_select)
          weights.append(weight)

      if (remaining_array - num_of_col>= 0).all():

        PE_array_index[count].append(indexes)
        PE_array_filter_num[count].append(filter_num + 1)
        n_columns = function_col_num(n_columns, num_of_col, remaining_array, full_array)
        f_s , weights_array = function_f_sel_and_weight(f_s, f_sels, weights_array, weights, remaining_array, full_array, f_weights.shape[0])
        remaining_array = remaining_array - num_of_col
        sel_mux_tr, result_col, en_adder_node = allocate_mux_tran (num_of_col, remaining_array, full_array, sel_mux_tr, result_col, en_adder_node, filter_num, f_weights.shape[0])

        num_of_col = np.zeros((N_ROWS_ARRAY,1))
        indexes = []
        f_sels = []
        weights = []

      else:
        PE_array_weights[count].append(weights_array)
        weights_array = np.zeros((N_ROWS_ARRAY,N_COLS_ARRAY))
        PE_array_weights.append([])
        num_of_columns[count].append(n_columns)
        n_columns = np.zeros((N_ROWS_ARRAY,N_COLS_ARRAY))
        num_of_columns.append([])
        PE_array_f_sel[count].append(f_s)
        f_s = np.zeros((N_ROWS_ARRAY,N_COLS_ARRAY))
        PE_array_f_sel.append([])
        sel_mux_transfer[count].append(sel_mux_tr)
        sel_mux_tr = np.zeros((N_ROWS_ARRAY,N_COLS_ARRAY))
        sel_mux_transfer.append([])
        result_columns[count].append(result_col)
        result_col = np.zeros((1,N_COLS_ARRAY))
        result_columns.append([])
        en_adder_node_all[count].append(en_adder_node)
        en_adder_node = np.zeros((N_ROWS_ARRAY,N_COLS_ARRAY))
        en_adder_node_all.append([])
        remaining_array = N_COLS_ARRAY * np.ones((N_ROWS_ARRAY, 1))
        f_s , weights_array = function_f_sel_and_weight(f_s, f_sels, weights, remaining_array, full_array, f_weights.shape[0])
        n_columns = function_col_num(n_columns, num_of_col, remaining_array, full_array)
        PE_array_index.append([])
        PE_array_filter_num.append([])
        count +=1
        PE_array_index[count].append(indexes)
        PE_array_filter_num[count].append(filter_num + 1)
        indexes =[]
        f_sels = []
        weights = []
        remaining_array = remaining_array - num_of_col
        sel_mux_tr, result_col, en_adder_node = allocate_mux_tran (num_of_col, remaining_array, full_array, sel_mux_tr, result_col, en_adder_node, filter_num, f_weights.shape[0])

    PE_array_weights[count].append(weights_array)
    weights_array = np.zeros((N_ROWS_ARRAY,N_COLS_ARRAY))
    PE_array_weights.append([])
    num_of_columns[count].append(n_columns)
    n_columns = np.zeros((N_ROWS_ARRAY,N_COLS_ARRAY))
    num_of_columns.append([])
    PE_array_f_sel[count].append(f_s)
    f_s = np.zeros((N_ROWS_ARRAY,N_COLS_ARRAY))
    PE_array_f_sel.append([])
    sel_mux_transfer[count].append(sel_mux_tr)
    sel_mux_tr = np.zeros((N_ROWS_ARRAY,N_COLS_ARRAY))
    sel_mux_transfer.append([])
    result_columns[count].append(result_col)
    result_col = np.zeros((1,N_COLS_ARRAY))
    result_columns.append([])
    en_adder_node_all[count].append(en_adder_node)
    en_adder_node = np.zeros((N_ROWS_ARRAY,N_COLS_ARRAY))
    en_adder_node_all.append([])
    remaining_array = N_COLS_ARRAY * np.ones((N_ROWS_ARRAY, 1))
    PE_array_index.append([])
    PE_array_filter_num.append([])
    count +=1
    weights = []
    indexes =[]
    f_sels = []
    num_of_col= np.zeros((N_ROWS_ARRAY,1))

  print("PE_array_f_sel")
  print(PE_array_f_sel)
  print("count")
  print(count)
  print(np.sum(abs(Q_f_weights) < pow(2,-8))/(f_weights.shape[0]*f_weights.shape[1]*f_weights.shape[2]*f_weights.shape[3]))
  print(1- ((count)*N_ROWS_ARRAY*N_COLS_ARRAY/(f_weights.shape[0]*f_weights.shape[1]*f_weights.shape[2]*f_weights.shape[3])))
  print("num_of_columns")
  print(num_of_columns)
  #print(PE_array_index)
  #print(PE_array_filter_num)
  print("sel_mux_transfer")
  print(sel_mux_transfer)
  #print(result_columns)
  print("en_adder_node_all")
  print(en_adder_node_all)
  print("PE_array_weights")
  print(PE_array_weights)
  print(np.array(en_adder_node_all[0][0][::-1]))

  # Example data: lists of NumPy arrays
  lists = [
    [np.array(en_adder_node_all[0][0][::-1])],  # First list
    [np.array(sel_mux_transfer[0][0][::-1])],  # Second list
    [np.array(num_of_columns[0][0][::-1])], # Third list
    [np.array(PE_array_f_sel[0][0][::-1])]
    ]

  # Define the bit lengths for each list (e.g., 2 bits for the first list, 4 bits for the second list)
  bit_lengths = [1, 2, 2, 2]

  # Run the function
  process_and_save(lists, bit_lengths, 'signal.txt')

  # Code to download the file in Google Colab
  files.download('signal.txt')

  lists = [
    [np.array(PE_array_weights[0][0][::-1])]
    ]

  # Define the bit lengths for each list (e.g., 2 bits for the first list, 4 bits for the second list)
  bit_lengths = [8]

  # Run the function
  process_and_save_weight(lists, bit_lengths, 'weight.txt')

  # Code to download the file in Google Colab
  files.download('weight.txt')


  return

In [22]:
def function_f_sel_and_weight(f_s, f_sels, weights_array, weights, remaining_array, full_array, filter_s):

  for ch in range(len(f_sels)):

    if len(f_sels[ch]) != 0:

      target_row = filter_s*ch

      for i in range(f_sels[ch].shape[0]) :

        for j in range(f_sels[ch].shape[1]):

          f_s[target_row + i, int(full_array[target_row + i, 0] - remaining_array[target_row + i, 0]) + j] = f_sels[ch][i, j]
          weights_array[target_row + i, int(full_array[target_row + i, 0] - remaining_array[target_row + i, 0]) + j] = weights[ch][i, j]

  return f_s, weights_array

In [23]:
def function_col_num(n_columns, num_of_col, remaining_array, full_array):

  for i in range(int(num_of_col.shape[0])):

    filled_array = full_array - remaining_array

    for j in range(int(num_of_col[i, 0])):

      n_columns[i, int(filled_array[i, 0]) + j] = num_of_col[i, 0]

  return n_columns

In [29]:
def filter_assign (f_weights_sorted_2D, near_zero_threshold , filter_s):

  num_non_zero_per_row = np.sum( np.abs(f_weights_sorted_2D) > near_zero_threshold, axis=1)
  max_num_non_zero_per_row = np.max(num_non_zero_per_row)
  num_of_col = max_num_non_zero_per_row * np.ones((f_weights_sorted_2D.shape[0],1))

  if max_num_non_zero_per_row != 0:

    index = np.full((f_weights_sorted_2D.shape[0],max_num_non_zero_per_row), np.nan)
    f_sel = (filter_s - 1)*np.ones((f_weights_sorted_2D.shape[0],max_num_non_zero_per_row))
    weight = np.zeros((f_weights_sorted_2D.shape[0],max_num_non_zero_per_row))
    count_r=0
    count_c=0

    for Row in range(f_weights_sorted_2D.shape[0]):

      for Col in range(f_weights_sorted_2D.shape[1]):

        if np.abs(f_weights_sorted_2D[Row, Col]) > near_zero_threshold :

          index[count_r, count_c] = Row*f_weights_sorted_2D.shape[0] + Col
          f_sel[count_r, count_c] = filter_s - 1 - (index[count_r, count_c] - count_r * f_weights_sorted_2D.shape[0] - count_c)
          weight[count_r, count_c] = f_weights_sorted_2D[Row, Col]
          count_c+=1

      count_r+=1
      count_c = 0

  else:

    index = []
    f_sel = []
    weight = []
  return num_of_col, index, f_sel, weight

In [25]:
def allocate_mux_tran (num_of_col, remaining_array, full_array, sel_mux_tr, result_col, en_adder_node, filter_num, filter_s):
  filled_array = full_array - remaining_array
  max_col = int(np.max(filled_array))
  if (max_col != 0):
    selected_col =  max_col - 1
    result_col[0, selected_col] = filter_num + 1
    for i in range(math.floor(full_array.shape[0]/filter_s)):

      if(num_of_col[(filter_s*(i+1)) - 1, 0] != 0):
        sel_mux_tr[(filter_s*(i+1)) - 1 , selected_col] =  max_col - filled_array [(filter_s*(i+1)) - 1, 0]
        en_adder_node [(filter_s*(i+1)) - 1 , selected_col] = 1
  return sel_mux_tr,  result_col, en_adder_node

In [26]:
import numpy as np
from google.colab import files

def process_and_save(lists, bit_lengths, filename='signal.txt'):
    """
    Process the lists of arrays and save the concatenated binary representations to a file.

    Args:
    lists (list of list of np.array): List of lists where each inner list contains NumPy arrays.
    bit_lengths (list of int): List of bit lengths for each list of arrays.
    filename (str): Name of the file to save the signal.
    """
    # Ensure bit_lengths is valid
    if len(lists) != len(bit_lengths):
        raise ValueError("Length of bit_lengths must match the number of lists.")

    # Determine the number of columns (assumes all arrays have the same number of columns)
    num_columns = len(lists[0][0][0]) if lists[0][0].ndim == 2 else len(lists[0][0])

    # Open the file for writing
    with open(filename, 'w') as file:
        # Iterate from the last column to the first column
        for col in range(num_columns-1, -1, -1):
            # Initialize a list to hold concatenated binary strings for this column
            concatenated_binaries = []

            # Process each list
            for i, (lst, bit_length) in enumerate(zip(lists, bit_lengths)):
                array = lst[0]  # Taking the first array in each list

                # Check if array is 1D or 2D
                if array.ndim == 1:
                    elements = array
                elif array.ndim == 2:
                    elements = array[:, col]
                else:
                    raise ValueError("Arrays must be either 1D or 2D.")

                # Process each element
                for element in elements:
                    # Convert the element to binary with specified bit length (unsigned)
                    binary_str = format(int(element), f'0{bit_length}b')
                    concatenated_binaries.append(binary_str)

            # Concatenate all binary strings
            concatenated_result = ''.join(concatenated_binaries)

            # Write the result to the file
            file.write(concatenated_result + '\n')

    print(f"Data has been processed and saved to {filename}")


In [27]:
import numpy as np
from google.colab import files

def process_and_save_weight(lists, bit_lengths, filename='weight.txt'):
    """
    Process the lists of arrays and save the concatenated binary representations to a file.

    Args:
    lists (list of list of np.array): List of lists where each inner list contains NumPy arrays.
    bit_lengths (list of int): List of bit lengths for each list of arrays.
    filename (str): Name of the file to save the output.
    """
    # Ensure bit_lengths is valid
    if len(lists) != len(bit_lengths):
        raise ValueError("Length of bit_lengths must match the number of lists.")

    # Determine the number of columns (assumes all arrays have the same number of columns)
    num_columns = len(lists[0][0][0]) if lists[0][0].ndim == 2 else len(lists[0][0])

    # Open the file for writing
    with open(filename, 'w') as file:
        # Iterate from the last column to the first column
        for col in range(num_columns-1, -1, -1):
            # Initialize a list to hold concatenated binary strings for this column
            concatenated_binaries = []

            # Process each list
            for i, (lst, bit_length) in enumerate(zip(lists, bit_lengths)):
                array = lst[0]  # Taking the first array in each list

                # Check if array is 1D or 2D
                if array.ndim == 1:
                    elements = array
                elif array.ndim == 2:
                    elements = array[:, col]
                else:
                    raise ValueError("Arrays must be either 1D or 2D.")

                # Process each element
                for element in elements:
                  # Convert float to integer (you can choose to round or truncate)
                  int_value = int(round(element))  # Use round() for rounding or int() for truncating

                  if int_value < 0:
                    # Calculate two's complement for negative numbers
                    binary_str = format((1 << bit_length) + int_value, f'0{bit_length}b')
                  else:
                    # Format positive numbers directly
                    binary_str = format(int_value, f'0{bit_length}b')
                  concatenated_binaries.append(binary_str)
            # Concatenate all binary strings
            concatenated_result = ''.join(concatenated_binaries)

            # Write the result to the file
            file.write(concatenated_result + '\n')

    print(f"Data has been processed and saved to {filename}")


In [ ]:
import numpy as np
print(np.max(np.zeros((2,4))) - np.ones((3,1)))

[[-1.]
 [-1.]
 [-1.]]


In [30]:
weights = np.array([[[[0, 0, 0],[0, 0, 0],[0, 0, 0]],[[1, 0, -3],[0, 0, 0],[5, 0, 1]],[[0, 3, -2],[0, 2, 1],[0, 1, 0]], [[0, 0, 0],[0, 0, 0],[0, 0, 0]],[[1, 0, -3],[0, 0, 0],[5, 0, 1]],[[0, 3, -2],[0, 2, 1],[0, 1, 0]]],
                   [[[1, 0, 2],[-1, 2, 3],[-2, 1, 1]],[[0, 0, 0],[0, 2, 0],[0, 0, 0]],[[3, 4, 0],[-1, -2, 0],[-3, -1, 1]],[[1, 0, 2],[-1, 2, 3],[-2, 1, 1]],[[0, 0, 0],[0, 2, 0],[0, 0, 0]],[[3, 4, 0],[-1, -2, 0],[-3, -1, 1]]],
                   [[[-1, 0, 1],[0, -3, 2],[1, 1, 0]],[[1, 4, 1],[-2, -3, 1],[1, -1, 0]],[[0, -2, 0],[1, 0, 0],[0, 0, -1]],[[-1, 0, 1],[0, -3, 2],[1, 1, 0]],[[1, 4, 1],[-2, -3, 1],[1, -1, 0]],[[0, -2, 0],[1, 0, 0],[0, 0, -1]]],
                   [[[0, -2, 3],[3, 2, 1],[0, 0, 5]],[[1, 1, 0],[2, 0, 3],[-1, 1, 0]],[[1, -1, 0],[0, -2, 0],[0, 0, -1]],[[0, -2, 3],[3, 2, 1],[0, 0, 5]],[[1, 1, 0],[2, 0, 3],[-1, 1, 0]],[[1, -1, 0],[0, -2, 0],[0, 0, -1]]]])
assign_PE(9,8, np.transpose(weights,(2,3,1,0)))

100
[2 1 3 0]
4
6
3
3
2
PE_array_f_sel
[[array([[2., 1., 2., 2., 1., 1., 1., 2.],
       [2., 2., 2., 1., 1., 2., 2., 2.],
       [2., 2., 2., 2., 2., 0., 2., 2.],
       [2., 1., 2., 2., 2., 2., 2., 2.],
       [2., 2., 1., 2., 2., 2., 2., 1.],
       [2., 1., 2., 2., 2., 2., 2., 2.],
       [1., 1., 2., 2., 2., 1., 2., 2.],
       [1., 1., 2., 2., 2., 2., 1., 2.],
       [1., 2., 2., 2., 2., 0., 0., 2.]])], [array([[2., 1., 2., 2., 1., 1., 1., 2.],
       [2., 2., 2., 1., 1., 2., 2., 2.],
       [2., 2., 2., 2., 2., 0., 2., 2.],
       [2., 1., 2., 2., 2., 2., 2., 2.],
       [2., 2., 1., 2., 2., 2., 2., 1.],
       [2., 1., 2., 2., 2., 2., 2., 2.],
       [1., 1., 2., 2., 2., 1., 2., 2.],
       [1., 1., 2., 2., 2., 2., 1., 2.],
       [1., 2., 2., 2., 2., 0., 0., 2.]])], []]
count
2
0.46296296296296297
0.33333333333333337
num_of_columns
[[array([[3., 3., 3., 2., 2., 3., 3., 3.],
       [3., 3., 3., 2., 2., 3., 3., 3.],
       [3., 3., 3., 2., 2., 3., 3., 3.],
       [2., 2., 1., 3.

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Data has been processed and saved to weight.txt


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
pip install torch torchvision

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torchvision import models

In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [ ]:
import torch
print("Is CUDA available: ", torch.cuda.is_available())

Is CUDA available:  True


In [ ]:
# 1. Load CIFAR-10 dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64,
                                         shuffle=False, num_workers=2)

# 2. Load the pretrained ResNet-18 model
model = models.resnet18(pretrained=True)

# 3. Modify the model
# Replace the first convolutional layer to accept 32x32 input images
model.conv1 = nn.Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)

# Replace the fully connected layer to output 10 classes for CIFAR-10
model.fc = nn.Linear(model.fc.in_features, 10)

# Move the model to the appropriate device
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model.to(device)

# 4. Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)



Files already downloaded and verified
Files already downloaded and verified


In [ ]:
# 5. Train the model
for epoch in range(10):  # Number of epochs
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:  # Print every 100 mini-batches
            print(f'[Epoch {epoch + 1}, Batch {i + 1}] loss: {running_loss / 100:.3f}')
            running_loss = 0.0

print('Finished Training')

# 6. Evaluate the model
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = model(images.to(device))
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels.to(device)).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total}%')

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader
# Function to prune weights
def prune_weights(model, pruning_amount=0.8):
    for name, module in model.named_modules():
        if isinstance(module, nn.Conv2d):
            # Number of weights in the Conv2d layer
            num_weights = module.weight.numel()

            # Calculate number of weights to prune
            num_prune = int(num_weights * pruning_amount)

            # Create a mask with the required number of weights set to zero
            weight = module.weight.data
            # Flatten weights to easily sample the desired amount of pruning
            weight_flat = weight.view(-1)
            # Get indices of weights to prune
            prune_indices = torch.topk(torch.abs(weight_flat), num_prune, largest=False).indices
            mask = torch.ones_like(weight_flat, dtype=torch.bool)
            mask[prune_indices] = False
            # Reshape the mask to match the original weight shape
            mask = mask.view(weight.size())

            # Apply the mask to weights
            module.weight.data *= mask

            # Optionally store the mask for future use
            module.weight_mask = mask

    return model

# Apply pruning
model = prune_weights(model, pruning_amount=0.8)

# Freeze all layers except the last layer
def freeze_except_last_layer(model):
    for name, param in model.named_parameters():
        if 'fc' in name:  # Assuming 'fc' is the name of the last layer in the model
            param.requires_grad = True
        else:
            param.requires_grad = False

# Freeze all layers except the last layer
freeze_except_last_layer(model)

# Define a simple transformation and dataset for fine-tuning
transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
])

dataset = CIFAR10(root='./data', train=True, transform=transform, download=True)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Define a simple fine-tuning loop
def fine_tune_last_layer(model, dataloader, epochs=10):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(
        [p for p in model.parameters() if p.requires_grad],
        lr=0.001,
        momentum=0.9
    )

    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, labels in dataloader:
          # Transfer inputs and labels to the same device as the model
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
        print(f'Epoch {epoch+1}, Loss: {running_loss / len(dataloader)}')

# Fine-tune the last layer
fine_tune_last_layer(model, dataloader, epochs=10)

# Print summary of parameters to verify the last layer is being updated
for name, param in model.named_parameters():
    if param.requires_grad:
        print(f'Parameter {name} is being updated.')

Files already downloaded and verified
Epoch 1, Loss: 1.7296254536469473
Epoch 2, Loss: 1.5455382372123343
Epoch 3, Loss: 1.4824859118964988
Epoch 4, Loss: 1.4487351546742102
Epoch 5, Loss: 1.4242327330361095
Epoch 6, Loss: 1.407959008857522
Epoch 7, Loss: 1.3958650290088934
Epoch 8, Loss: 1.380197656741908
Epoch 9, Loss: 1.3720881578560755
Epoch 10, Loss: 1.3652122113770273
Parameter fc.weight is being updated.
Parameter fc.bias is being updated.


In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torchvision import models
import torch.optim as optim
def apply_random_pruning(module, pruning_amount):
    """
    Apply random pruning to Conv2D and Linear layers in the model.
    """
    if isinstance(module, (nn.Conv2d, nn.Linear)): #
        with torch.no_grad():
            weight = module.weight.data
            mask = torch.rand_like(weight) < pruning_amount
            module.weight.data *= mask
            module.register_buffer('mask', mask)  # Register mask as a buffer
def freeze_pruned_weights(model):
    """
    Freeze weights that have been pruned.
    """
    #for name, param in model.named_parameters():
        #if 'mask' in name:
            # Get the corresponding weight tensor and its mask
            #layer_name = name.replace('.mask', '')
            #weight = dict(model.named_parameters())[layer_name]
            #if weight.requires_grad:
                #weight.requires_grad = False  # Freeze pruned weights
    for name, param in model.named_parameters():
        if 'fc' in name:  # Assuming 'fc' is the name of the last layer in the model
            param.requires_grad = True
        else:
            param.requires_grad = False
def train_model(model, trainloader, criterion, optimizer, epochs):
    model.train()

    for epoch in range(epochs):
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            inputs, labels = inputs.to('cuda' if torch.cuda.is_available() else 'cpu'), labels.to('cuda' if torch.cuda.is_available() else 'cpu')

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            if i % 100 == 99:
                print(f'[Epoch {epoch + 1}] loss: {running_loss / 100:.3f}')
                running_loss = 0.0
# Apply pruning
pruning_amount = 0.8  # Prune 80% of weights
for module in model.modules():
    apply_random_pruning(module, pruning_amount)

# Freeze pruned weights
freeze_pruned_weights(model)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001, momentum=0.9)

# Retrain the model
train_model(model, trainloader, criterion, optimizer, epochs=10)

# Evaluate the model
def evaluate(model, testloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            outputs = model(images.to('cuda' if torch.cuda.is_available() else 'cpu'))
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels.to('cuda' if torch.cuda.is_available() else 'cpu')).sum().item()
    print(f'Accuracy of the network on the 10000 test images: {100 * correct // total}%')

evaluate(model, testloader)

[Epoch 1] loss: 2.267
[Epoch 1] loss: 2.030
[Epoch 1] loss: 1.960
[Epoch 1] loss: 1.908
[Epoch 1] loss: 1.898
[Epoch 1] loss: 1.874
[Epoch 1] loss: 1.861
[Epoch 2] loss: 1.793
[Epoch 2] loss: 1.816
[Epoch 2] loss: 1.841
[Epoch 2] loss: 1.818
[Epoch 2] loss: 1.805
[Epoch 2] loss: 1.826
[Epoch 2] loss: 1.818
[Epoch 3] loss: 1.812
[Epoch 3] loss: 1.763
[Epoch 3] loss: 1.818
[Epoch 3] loss: 1.794
[Epoch 3] loss: 1.783
[Epoch 3] loss: 1.795
[Epoch 3] loss: 1.794
[Epoch 4] loss: 1.793
[Epoch 4] loss: 1.759
[Epoch 4] loss: 1.770
[Epoch 4] loss: 1.794
[Epoch 4] loss: 1.789
[Epoch 4] loss: 1.796
[Epoch 4] loss: 1.792
[Epoch 5] loss: 1.754
[Epoch 5] loss: 1.788
[Epoch 5] loss: 1.781
[Epoch 5] loss: 1.779
[Epoch 5] loss: 1.763
[Epoch 5] loss: 1.766
[Epoch 5] loss: 1.777
[Epoch 6] loss: 1.749
[Epoch 6] loss: 1.772
[Epoch 6] loss: 1.761
[Epoch 6] loss: 1.800
[Epoch 6] loss: 1.780
[Epoch 6] loss: 1.784
[Epoch 6] loss: 1.779
[Epoch 7] loss: 1.776
[Epoch 7] loss: 1.781
[Epoch 7] loss: 1.758
[Epoch 7] 

In [ ]:
import torch
import numpy as np

def model_weights_to_numpy(model):
    """
    Convert model weights to a dictionary of NumPy arrays.
    """
    weights_dict = {}
    for name, param in model.named_parameters():
        # Convert to NumPy array and store in the dictionary
        weights_dict[name] = param.detach().cpu().numpy()
    return weights_dict

# Assuming 'model' is your trained and pruned model
weights_dict = model_weights_to_numpy(model)

# Example: Access weights for a specific layer
conv1_weights = weights_dict['conv1.weight']
print(conv1_weights.shape)

(64, 3, 3, 3)


In [ ]:
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:

print(weights_dict['layer3.1.conv1.weight'].shape)
assign_PE(18,36, np.transpose(weights_dict['layer3.1.conv1.weight'],(2,3,1,0)))

(256, 256, 3, 3)
471859
[ 54   7  51 150 254  87  19  65 235 136  95 160 227 153 173  70 248  29
 216  88  48  77 161 208 214 184 232 191  78 225 228  34 110 199 128  80
 147 138 252 250 188 192  55 240 157 166   4 132 121 125 189  35  21 200
 205 142  14 118 181 255 245 194  31 179  40 190 242 124 193   1  93  25
 212 238 154  85  46  75  86 170 247 145  28  24 217 111 164 186 244 178
 197 218 202 133  23  72  30   9   5 169  68 220 139  22 168 144 201 229
  13 167 224  82 129  76  59 115  39 100 233 108  11 165 231 120 130  98
  92 196 109  99  32 185  96  41  47 126 221 249 230 251 183 114  16 241
 162  18 137  33 134 253 237 180 236 222 107 223  79  74 182 101  53 119
 102  81  50  49 148 239 140  17 151  61  67 246 198 116 159 210  94  12
  91 143 112  37 195 117  44  20 146 152 106 209 123 103  69   8 156 219
 206  45  57 213  71  97 105 135 175  15  84   3 203  38 204 177  62 127
 207  56 243  26  64 155 163  10  52 104  36 122 158  42  60 215   6 149
   0 171  66  89 176 141 11

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torchvision import models

# Define random pruning function
def random_pruning(module, amount):
    """
    Apply random pruning to a Conv2D or Linear layer.
    """
    if isinstance(module, (nn.Conv2d, nn.Linear)):
        weight = module.weight.data
        # Create a mask with the same shape as the weights, where a fraction is zeroed
        mask = torch.rand_like(weight) < amount
        # Apply mask to weights
        module.weight.data.mul_(mask)
# Function for training
def train(model, trainloader, criterion, optimizer, epochs):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            inputs, labels = inputs.to('cuda' if torch.cuda.is_available() else 'cpu'), labels.to('cuda' if torch.cuda.is_available() else 'cpu')

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            if i % 100 == 99:  # Print every 100 mini-batches
                print(f'[Epoch {epoch + 1}] loss: {running_loss / 100:.3f}')
                running_loss = 0.0
        # Optionally save checkpoints

# Function for evaluation
def evaluate(model, testloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            outputs = model(images.to('cuda' if torch.cuda.is_available() else 'cpu'))
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels.to('cuda' if torch.cuda.is_available() else 'cpu')).sum().item()
    print(f'Accuracy of the network on the 10000 test images: {100 * correct // total}%')

# Iterative random pruning and fine-tuning
prune_amounts = [0.2, 0.4, 0.5]  # Define pruning amounts for each iteration

for amount in prune_amounts:
    print(f"Applying {amount*100}% random pruning")
    for module in model.modules():
        random_pruning(module, amount)
    train(model, trainloader, criterion, optimizer, epochs=5)  # Fine-tune the model
    evaluate(model, testloader)

print("Finished Iterative Random Pruning and Fine-Tuning")

Applying 20.0% random pruning
[Epoch 1] loss: 1.874
[Epoch 1] loss: 1.479
[Epoch 1] loss: 1.369
[Epoch 1] loss: 1.254
[Epoch 1] loss: 1.214
[Epoch 1] loss: 1.159
[Epoch 1] loss: 1.071
[Epoch 2] loss: 0.933
[Epoch 2] loss: 0.922
[Epoch 2] loss: 0.887
[Epoch 2] loss: 0.863
[Epoch 2] loss: 0.826
[Epoch 2] loss: 0.813
[Epoch 2] loss: 0.792
[Epoch 3] loss: 0.627
[Epoch 3] loss: 0.620
[Epoch 3] loss: 0.677
[Epoch 3] loss: 0.656
[Epoch 3] loss: 0.636
[Epoch 3] loss: 0.640
[Epoch 3] loss: 0.637
[Epoch 4] loss: 0.437
[Epoch 4] loss: 0.475
[Epoch 4] loss: 0.479
[Epoch 4] loss: 0.485
[Epoch 4] loss: 0.492
[Epoch 4] loss: 0.499
[Epoch 4] loss: 0.510
[Epoch 5] loss: 0.295
[Epoch 5] loss: 0.312
[Epoch 5] loss: 0.343
[Epoch 5] loss: 0.397
[Epoch 5] loss: 0.379
[Epoch 5] loss: 0.382
[Epoch 5] loss: 0.369
Accuracy of the network on the 10000 test images: 77%
Applying 40.0% random pruning
[Epoch 1] loss: 1.439
[Epoch 1] loss: 1.119
[Epoch 1] loss: 0.998
[Epoch 1] loss: 0.896
[Epoch 1] loss: 0.837
[Epoch

In [ ]:
for module in model.modules():
        random_pruning(module, 0.5)
evaluate(model, testloader)

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Accuracy of the network on the 10000 test images: 10%


In [ ]:
import torch
import numpy as np

def model_weights_to_numpy(model):
    """
    Convert model weights to a dictionary of NumPy arrays.
    """
    weights_dict = {}
    for name, param in model.named_parameters():
        # Convert to NumPy array and store in the dictionary
        weights_dict[name] = param.detach().cpu().numpy()
    return weights_dict

# Assuming 'model' is your trained and pruned model
weights_dict = model_weights_to_numpy(model)

# Example: Access weights for a specific layer
conv1_weights = weights_dict['conv1.weight']
print(conv1_weights.shape)

(64, 3, 3, 3)


In [ ]:
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:

print(weights_dict['layer4.0.conv2.weight'].shape)
assign_PE(9,16, np.transpose(weights_dict['layer4.0.conv2.weight'],(2,3,1,0)))

(512, 512, 3, 3)
1945595
[319 341 372 412  22 148 257 200  41 204 117 161 202 226 505 316 310 163
  47 490 483 382 221  31 182 485 193 349  25 249 203 147 180 318 343 283
 178  96 484 460 287 329  66  13 122 208 500 137 184 214 192 142 457 307
 424 415 404 211 482  21 508 440 284 323 210 265  17  18 239 262 260 228
 378 171 274  71  27 276 362 437 357 101 449 474 309 273 481 369 301 314
 294   7 166 198 277 196 452  84 414 176  30 220 113 290 431  50 300 388
 209 128 436 270 386  68 399 336 107 394 306 393 330 327 502 443 498  87
 344  69  94 338   4  90 320 243  63 250 115  83 322 373 175 305 233   9
 374 491 405 342 469 488 213 454 397   6 416 104 462 150 422 325 170 479
 419 109 446 103 247 370 458  16 114 297 242  19  42 381 312  53  36 266
 216  49   2  26 466 456 358 164 377 495 156 217 126 291 279  88  38  11
 235 475 146 435 254 339 385  86  55 501 275   1 365  64 231 430  20  23
 442 263  77 174  75 256  44 347  98 238 236 255 186  85 224 286 335 364
   0 434 222 199 223 447 3

In [ ]:
print( np.transpose(weights,(2,3,1,0))[:,:,1,0])


[[ 1  0 -3]
 [ 0  0  0]
 [ 5  0  1]]


In [ ]:
weights= model.conv1.weight.data
print(weights.shape)
assign_PE(14,32, np.transpose(weights,(2,3,1,0)))

torch.Size([64, 3, 7, 7])


TypeError: max() received an invalid combination of arguments - got (out=NoneType, axis=NoneType, ), but expected one of:
 * ()
 * (Tensor other)
 * (int dim, bool keepdim = False)
      didn't match because some of the keywords were incorrect: out, axis
 * (name dim, bool keepdim = False)
      didn't match because some of the keywords were incorrect: out, axis


In [ ]:
import torch
import numpy as np

# Load PyTorch model
pt_model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=False)
pt_model.load_state_dict(torch.load('resnet50_pruned_70_state_dict.pth'))

# Extract weights and save as NumPy arrays
weights = {}
for name, param in pt_model.named_parameters():
    weights[name] = param.detach().cpu().numpy()

# Save weights as NumPy files
np.save('weights_conv1.npy', weights['conv1.weight'])
# Repeat for other layers...

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0
<ipython-input-38-2925d8b3bfe7>:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  pt_model.load_state_dict(

In [ ]:
from google.colab import files

In [ ]:
uploaded = files.upload()


Saving resnet50_pruned_70_state_dict.pth to resnet50_pruned_70_state_dict.pth
